In [0]:
dbutils.fs.ls("/mnt/storage/load/dataset-ml")

[FileInfo(path='dbfs:/mnt/storage/load/dataset-ml/LoanApprovalPrediction.csv', name='LoanApprovalPrediction.csv', size=37135, modificationTime=1705526513000),
 FileInfo(path='dbfs:/mnt/storage/load/dataset-ml/Pacientes2.csv', name='Pacientes2.csv', size=28555, modificationTime=1705410619000),
 FileInfo(path='dbfs:/mnt/storage/load/dataset-ml/Pacientes_nuevos.csv', name='Pacientes_nuevos.csv', size=159, modificationTime=1705411019000)]

In [0]:
from pyspark.sql.functions import col, expr, regexp_replace, monotonically_increasing_id, lit, to_date, when, mean, round
from pyspark.sql.types import StructType, StructField, DateType, IntegerType, StringType, TimestampType

In [0]:
path = "/mnt/storage/load/dataset-ml/LoanApprovalPrediction.csv"
df_prestamos = spark.read.csv(path, header = True, inferSchema = True, sep = ",")

display(df_prestamos)

Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
LP001002,Male,No,0,Graduate,No,5849,0.0,null,360,1,Urban,Y
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128,360,1,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66,360,1,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120,360,1,Urban,Y
LP001008,Male,No,0,Graduate,No,6000,0.0,141,360,1,Urban,Y
LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267,360,1,Urban,Y
LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95,360,1,Urban,Y
LP001014,Male,Yes,3,Graduate,No,3036,2504.0,158,360,0,Semiurban,N
LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168,360,1,Urban,Y
LP001020,Male,Yes,1,Graduate,No,12841,10968.0,349,360,1,Semiurban,N


In [0]:
df_prestamos.printSchema()

root
 |-- Loan_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Married: string (nullable = true)
 |-- Dependents: integer (nullable = true)
 |-- Education: string (nullable = true)
 |-- Self_Employed: string (nullable = true)
 |-- ApplicantIncome: integer (nullable = true)
 |-- CoapplicantIncome: double (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- Loan_Amount_Term: integer (nullable = true)
 |-- Credit_History: integer (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Loan_Status: string (nullable = true)



In [0]:
##ir a bronze
df_prestamos.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save("/mnt/storage/layers/bronze/ml/prestamosBronze")

In [0]:
df_prestamosP = df_prestamos.toPandas()
df_prestamosP.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents           12
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           21
Loan_Amount_Term     14
Credit_History       49
Property_Area         0
Loan_Status           0
dtype: int64

In [0]:
df_prestamosSilver = df_prestamos.dropna(subset=["Dependents", "LoanAmount", "Loan_Amount_Term", "Credit_History"])
display(df_prestamosSilver)
df_prestamosSilver.count()

Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128,360,1,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66,360,1,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120,360,1,Urban,Y
LP001008,Male,No,0,Graduate,No,6000,0.0,141,360,1,Urban,Y
LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267,360,1,Urban,Y
LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95,360,1,Urban,Y
LP001014,Male,Yes,3,Graduate,No,3036,2504.0,158,360,0,Semiurban,N
LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168,360,1,Urban,Y
LP001020,Male,Yes,1,Graduate,No,12841,10968.0,349,360,1,Semiurban,N
LP001024,Male,Yes,2,Graduate,No,3200,700.0,70,360,1,Urban,Y


505

In [0]:
##ir a Silver
df_prestamosSilver.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save("/mnt/storage/layers/silver/ml/prestamosSilver")

In [0]:
meam_dependents = df_prestamos.select(round(mean(col('Dependents'))).cast('integer')).collect()[0][0]
display(meam_dependents)

1

In [0]:
meam_loan_amount = df_prestamos.select(round(mean(col('LoanAmount'))).cast('integer')).collect()[0][0]

display(meam_loan_amount)

145

In [0]:
meam_loan_term = df_prestamos.select(round(mean(col('Loan_Amount_Term'))).cast('integer')).collect()[0][0]

display(meam_loan_term)

342

In [0]:
meam_credit_history = df_prestamos.select(round(mean(col('Credit_History'))).cast('integer')).collect()[0][0]

display(meam_credit_history)

1

In [0]:
df_prestamosGold = df_prestamos

df_prestamosGold = df_prestamosGold.withColumn('Dependents', when(col('Dependents').isNull(), lit(dependents_me)).otherwise(col('Dependents')))
df_prestamosGold = df_prestamosGold.withColumn('LoanAmount', when(col('LoanAmount').isNull(), lit(loan_amount_me)).otherwise(col('LoanAmount')))
df_prestamosGold = df_prestamosGold.withColumn('Loan_Amount_Term', when(col('Loan_Amount_Term').isNull(), lit(loan_term_me)).otherwise(col('Loan_Amount_Term')))
df_prestamosGold = df_prestamosGold.withColumn('Credit_History', when(col('Credit_History').isNull(), lit(credit_history_me)).otherwise(col('Credit_History')))

display(df_prestamosGold)

Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
LP001002,Male,No,0,Graduate,No,5849,0.0,145,360,1,Urban,Y
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128,360,1,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66,360,1,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120,360,1,Urban,Y
LP001008,Male,No,0,Graduate,No,6000,0.0,141,360,1,Urban,Y
LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267,360,1,Urban,Y
LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95,360,1,Urban,Y
LP001014,Male,Yes,3,Graduate,No,3036,2504.0,158,360,0,Semiurban,N
LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168,360,1,Urban,Y
LP001020,Male,Yes,1,Graduate,No,12841,10968.0,349,360,1,Semiurban,N


In [0]:
#ir a Gold
df_prestamosGold.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save("/mnt/storage/layers/gold/ml/prestamosGold")